In [148]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

In [149]:
device = torch.device("cpu")

# if torch.backends.mps.is_available() and torch.backends.mps.is_built():
#     device = torch.device("mps")

device

device(type='cpu')

In [150]:
txt = open('shakespeare.txt', 'r').read()
len(txt)

5447119

In [151]:
txt[0:500]

"                     1\n  From fairest creatures we desire increase,\n  That thereby beauty's rose might never die,\n  But as the riper should by time decease,\n  His tender heir might bear his memory:\n  But thou contracted to thine own bright eyes,\n  Feed'st thy light's flame with self-substantial fuel,\n  Making a famine where abundance lies,\n  Thy self thy foe, to thy sweet self too cruel:\n  Thou that art now the world's fresh ornament,\n  And only herald to the gaudy spring,\n  Within thine own bud"

In [152]:
chars = list(set(txt))
chars.sort()

ctoi = {c:i for i, c in enumerate(chars)}
itoc = {i:c for i, c in enumerate(chars)}
vocab_size = len(chars)

print("".join(chars))
print(vocab_size)


 !"&'(),-.0123456789:;<>?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_`abcdefghijklmnopqrstuvwxyz|}
84


In [153]:
i =  math.floor(0.9 * len(txt))
train_txt = txt[0:i]
valid_txt = txt[i+1:]

len(train_txt), len(valid_txt)

(4902407, 544711)

In [154]:
train_tkns = [ctoi[c] for c in train_txt]
valid_tkns = [ctoi[c] for c in valid_txt]

In [155]:
block_size = 8
batch_size = 32

def txt_to_token(t):
    return [ctoi[c] for c in t]
    
# (B, L)
def random_batch():
    xi = torch.randint(0, len(train_tkns)-block_size, (batch_size,))
    x = torch.tensor([train_tkns[i:i+block_size] for i in xi], device=device)
    y = torch.tensor([train_tkns[i+1:i+block_size+1] for i in xi], device=device)
    
    return x, y

x, y = random_batch()
x.shape

torch.Size([32, 8])

In [156]:
x[0]

tensor([ 1, 56,  1, 58, 67, 70, 78, 69])

In [157]:
@torch.no_grad()
def eval_split(split, model):
    model.eval()
    
    tkn = train_tkns if split == "train" else valid_tkns
    tkn = torch.tensor(tkn)

    bsize = len(tkn) // block_size
    x = tkn[0:bsize*block_size]
    y = tkn[1:bsize*block_size+1]
    x = x.view(bsize, block_size) # (B, L)
    y = y.view(bsize, block_size) # (B, L)
    
    x = x.to(device)
    y = y.to(device)
    
    logits = model(x) # (B, L, C)
    B, L, C = logits.shape
    loss = F.cross_entropy(logits.view(B*L, C), y.view(B*L))
    
    model.train()
    return loss.item()

In [158]:
@torch.no_grad()
def sample(model):
    model.eval()
    
    max_len = 500
    tks = [0]*block_size

    for i in range(max_len):
        ctx = torch.tensor(tks[i:i+block_size]) # (L)
        ctx = ctx.view(1, -1) # (B, L)

        logits = model(ctx) # (B, L, C)
        probs = F.softmax(logits, dim=2) # (B, L, C)
        probs = probs[0,-1,:] # (C), # the last in the sequence is the newly generated
        yi = torch.multinomial(probs, 1)
        tks.append(yi.item())

    chars = [itoc[t] for t in tks]
    model.train()
    return "".join(chars)

In [159]:
def fit(model):
    optim = torch.optim.Adam(model.parameters())
    model.train()
    
    for i in range(50000):
        optim.zero_grad()

        xb, yb = random_batch()
        logits = model(xb) # (B, L, C)

        B, L, C = logits.shape
        loss = F.cross_entropy(logits.view(B*L, C), yb.view(B*L))

        loss.backward()
        optim.step()

        if i % 5000 == 0:
            print(f"{loss.item():.4f}")

In [160]:
class MultiHeadAttension(nn.Module):    
    
    def __init__(self, head_num, head_size, in_size, out_size):
        super().__init__()
        
        self.head_size = head_size
        self.head_num = head_num        
        self.attn = nn.Linear(in_size, 3 * head_num * head_size, bias=False)
        self.ffn = nn.Linear(head_num * head_size, out_size, bias=False)
    
        
    # x: (B, L, C)  
    # return: (B, L, C')
    def forward(self, x):
        B, L, C = x.shape
        
        z = self.attn(x) # (B, L, 3 * hn * hs)
        k, q, v = torch.split(z, self.head_num * self.head_size, dim=2) # (B, L, hn * hs)
        
        k = k.view(B, L, self.head_num, self.head_size).permute(0, 2, 1, 3) # (B, hn, L, hs)
        q = q.view(B, L, self.head_num, self.head_size).permute(0, 2, 1, 3)
        v = v.view(B, L, self.head_num, self.head_size).permute(0, 2, 1, 3)
        
        q = q.permute(0, 1, 3, 2) # (B, hn, hs, L)
        attn = (k @ q) / self.head_size**0.5 # (B, hn, L, L)
        mask = torch.tril(torch.ones(L, L)) == 0
        mask = mask.to(device)
        attn = attn.masked_fill(mask, -float('inf')) # (B, hn, L, L)
        attn = F.softmax(attn, dim=3)
        
        y = attn @ v # (B, hn, L, hs)
        y = y.permute(0, 2, 1, 3) # (B, L, hn, hs)
        y = y.contiguous().view(B, L, -1) # (B, L, hn * hs)
        y = self.ffn(y) # (B, L, C)
        
        return y 
    
        
x = torch.randn(2, block_size, 9) # (B, L, C)
x = x.to(device)
mh = MultiHeadAttension(5, 3, 9, 7)
mh = mh.to(device)
mh(x).shape

torch.Size([2, 8, 7])

In [161]:
class MLP(nn.Module):
    
    def __init__(self, in_size, out_size):
        super().__init__()
        self.linear1 = nn.Linear(in_size, out_size)
        self.linear2 = nn.Linear(out_size, out_size)
    
    # (B, L, C)
    def forward(self, x):
        y = self.linear1(x)
        y = torch.relu(y)
        y = self.linear2(y)
        
        return y

In [162]:
class Block(nn.Module):    
    
    def __init__(self, emb_size, head_size):
        super().__init__()
        
        assert emb_size % head_size == 0
        head_num = emb_size // head_size
        
        self.mha = MultiHeadAttension(head_num, 
                                      head_size, 
                                      in_size=emb_size, 
                                      out_size=emb_size)
        self.lnorm1 = nn.LayerNorm(emb_size)
        self.lnorm2 = nn.LayerNorm(emb_size)
        self.ffn = MLP(emb_size, emb_size)
        
        
    # x: (B, L, emb)
    def forward(self, x):
        y = self.mha(x) + x
        y = self.lnorm1(y)
        y = self.ffn(y) + y
        y = self.lnorm2(y)
        return y
    
# x = torch.randn(3, 4, 10)
# b = Block(10, 2)
# b(x)

In [163]:
# return (L, C)
def pos_encoding(x):
    B, L, C = x.shape
    pos = torch.arange(0, L).view(-1, 1) # (L, 1)
    div = 2 * torch.arange(0, C) / C # (C)
    div = torch.pow(10000, div) # (C)
    e = pos / div
    e[1, 1]

    pe = torch.zeros(L, C)
    pe[:,0::2] = torch.sin(e[:,0::2])
    pe[:,1::2] = torch.cos(e[:,1::2])
    
    pe = pe.to(device)
    return pe

In [164]:
emb_size = 100
head_size = 20

class Transformer(nn.Module):    
    
    def __init__(self):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, emb_size)
        self.blocks = nn.Sequential(
            Block(emb_size, head_size),
            Block(emb_size, head_size)
        )
        self.linear = nn.Linear(emb_size, vocab_size)

    # (B, L) -> (B, L, C)
    def forward(self, x):
        y = self.embed(x) # (B, L, emb)
        y = y + pos_encoding(y) # (B, L, emb)
        y = self.blocks(y) # (B, L, emb)
        y = self.linear(y) # (B, L, vocab)
        
        return y

In [165]:
tf_model = Transformer()
tf_model = tf_model.to(device)

count = sum([p.numel() for p in tf_model.parameters()])
print(f"total parameter: {count}")

total parameter: 138084


In [166]:
fit(tf_model)

4.5527
1.9368
1.9549
1.7029
1.8572
1.6687
1.8032
1.6225
1.7311
1.7076


In [174]:
tr_loss = eval_split("train", tf_model)
va_loss = eval_split("valid", tf_model)

print(f"train: {tr_loss:.4f}")
print(f"valid: {va_loss:.4f}")

train: 1.7155
valid: 1.7952


In [173]:
print(sample(tf_model))










SCENE . Pas what Firchy. This have I for the should no might.
  MRS. Rance thrack'd
    He to almm your swattly neit will again, wink
    Not such 'that
    I do thingy your keespech of what is the ford pay; I had bounds his monio couxshould wing.
                               Exit Along she keeps him as it is may a bring the weash steek that not, they and time follain tongain! Bart; give of bland.
  SIR OR MEMBERSHIP.>>



ACT I. Fallowar.
  SURLOS. Take to your lives strike ire, Own thee; do


## Log

- Bi-gram: 2.4716, 2.4755
- Single-head attention: 2.3899, 2.4041
- Multi-head attention, single layer: 2.0820, 2.1165
    - block_size = 8
    - hidden_size = 100
    - head_size = 20
    - head_num = 5
    
- Multi-head attention, single layer, positional encoding: 1.8575, 1.9216
- 2-layer transformer (with everything, MHA, positional encoding, layer norm): 1.7155, 1.7952